In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from pathlib import Path
import torch
import random
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from torchvision import transforms

In [ ]:
import wandb
wandb.init(project="osic-pulmonary-pytorch-resnet-50")

In [ ]:
from lib import common

In [ ]:
pd.options.mode.chained_assignment = None

### Path

In [ ]:
path = Path('/kaggle/osic_pulmonary')
assert path.exists()

### Read Data

In [ ]:
train_df, test_df, submission_df = common.read_data(path)

#### Feature generation

In [ ]:
submission_df = common.prepare_submission(submission_df, test_df)

In [ ]:
submission_df[((submission_df['Patient'] == 'ID00419637202311204720264') & (submission_df['Weeks'] == 6))].head(25)

In [ ]:
def adapt_percent_in_submission():
    previous_match = None
    for i, r in submission_df.iterrows():
        in_training = train_df[(train_df['Patient'] == r['Patient']) & (train_df['Weeks'] == r['Weeks'])]
        if(len(in_training) > 0):
            previous_match = in_training['Percent'].item()
            submission_df.iloc[i, submission_df.columns.get_loc('Percent')] = previous_match
        elif previous_match is not None:
            submission_df.iloc[i, submission_df.columns.get_loc('Percent')] = previous_match

In [ ]:
adapt_percent_in_submission()

In [ ]:
test_df[test_df['Patient'] == 'ID00419637202311204720264']

In [ ]:
train_df[train_df['Patient'] == 'ID00419637202311204720264']

In [ ]:
submission_df[submission_df['Patient'] == 'ID00419637202311204720264'].head(10)

Adding missing values

In [ ]:
train_df['WHERE'] = 'train'
test_df['WHERE'] = 'val'
submission_df['WHERE'] = 'test'
data = train_df.append([test_df, submission_df])

In [ ]:
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

In [ ]:
base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base

In [ ]:
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
data['base_week'] = data['base_week']
del base

In [ ]:
data[data['Patient'] == 'ID00421637202311550012437']

In [ ]:
COLS = ['Sex','SmokingStatus'] #,'Age'
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)

In [ ]:
data[data['Patient'] == 'ID00421637202311550012437']

In [ ]:
def normalize(df:pd.DataFrame, cont_names, target_names):
    "Compute the means and stds of `self.cont_names` columns to normalize them."
    means, stds = {},{}
    for n, t in zip(cont_names, target_names):
        means[n], stds[n] = df[n].mean(), df[n].std()
        df[t] = (df[n]-means[n]) / (1e-7 + stds[n])

normalize(data, ['Age','min_FVC','base_week','Percent'], ['age','BASE','week','percent'])
FE += ['age','percent','week','BASE']

In [ ]:
train_df = data.loc[data.WHERE=='train']
test_df = data.loc[data.WHERE=='val']
submission_df = data.loc[data.WHERE=='test']
del data

In [ ]:
train_df.sort_values(['Patient', 'Weeks']).head(10)

In [ ]:
X = train_df[FE]
X.head(15)

In [ ]:
y = train_df['FVC']
y.shape

#### Seed

In [ ]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    
seed_everything(42)

### Read Images

In [ ]:
NUMBER_IMAGES = 10
wandb.config.num_images = NUMBER_IMAGES

In [ ]:
import glob, os

def read_png(patient, ds_type='train', limit=1):
    current_folder = path/f"{ds_type}/{patient}"
    os.chdir(current_folder)
    img_files = glob.glob("*.png")
    pil_images = []
    for i in range(limit):
        if len(img_files) > i:
            img_file = img_files[i]
        else:
            raise Exception(f"Not enough images in folder {current_folder}. Only {len(img_files)} found and at least {i + 1} expected.")
        full_path = path/f"{ds_type}/{patient}/{img_file}"
        image_tensor = transforms.functional.to_tensor(Image.open(full_path))
        image_tensor = image_tensor[0].unsqueeze(0)
        pil_images.append(image_tensor)
    return torch.cat(pil_images)

read_png('ID00015637202177877247924', limit = 1).shape, read_png('ID00015637202177877247924', limit = NUMBER_IMAGES).shape

In [ ]:
!ls /kaggle/osic_pulmonary/train/ID00248637202266698862378/*.png | wc -l

In [ ]:
torch.cat([read_png(patient) for patient in ['ID00419637202311204720264', 'ID00421637202311550012437']]).shape

### Create Dataset

In [ ]:
class TabularImageDataset(Dataset):
    
    def __init__(self, x, y, patients, image_path='train'):
        self.x, self.y = torch.tensor(x.values, dtype=torch.float32), torch.tensor(y.values, dtype=torch.float32)
        self.patients = patients.values
        self.image_path = image_path
        assert(len(self.x) == len(self.y))
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        patient = self.patients[i]
        return self.x[i], self.y[i], read_png(patient, ds_type=self.image_path, limit = NUMBER_IMAGES)
    
    def __repr__(self):
        return f'x: {self.x.shape} y: {self.y.shape}, patients: {len(self.patients)}'

In [ ]:
def create_dl(X, y, patients, image_path='train', batch_size=64, num_workers=10, shuffle=True):
    ds = TabularImageDataset(X, y, patients, image_path)
    return DataLoader(ds, batch_size, shuffle=shuffle, num_workers=num_workers)

In [ ]:
sample_dl = create_dl(X, y, train_df['Patient'], image_path='train')
x_sample, y_sample, image_sample = next(iter(sample_dl))
x_sample.shape, y_sample.shape, image_sample.shape[1]

### Prepare neural network

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def move_to_dev(x, y, img=None):
    x = x.to(device)
    y  = y.to(device)
    if img is not None:
        img = img.to(device)
    else:
        img = None
    return x, y, img

In [ ]:
C1, C2 = torch.tensor(70, dtype=torch.float32), torch.tensor(1000, dtype=torch.float32)
C1, C2, _ = move_to_dev(C1, C2)
q = torch.tensor([0.2, 0.50, 0.8]).float().to(device)

def score(y_true, y_pred):
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = torch.max(sigma, C1)
    delta = torch.abs(y_true[:, 0] - fvc_pred)
    delta = torch.min(delta, C2)
    sq2 = torch.sqrt(torch.tensor(2.))
    metric = (delta / sigma_clip)*sq2 + torch.log(sigma_clip* sq2)
    return torch.mean(metric)

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    e = y_true - y_pred
    v = torch.max(q*e, (q-1)*e)
    return torch.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        y_true = y_true.unsqueeze(1)
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss

In [ ]:
def conv2d(ni, nf, ks=3, stride=2, add_relu=True, padding=None):
    if padding is None:
        padding = ks//2
    if add_relu:
        return nn.Sequential(nn.Conv2d(ni, nf, ks, padding=padding, stride=stride), nn.ReLU(), nn.BatchNorm2d(nf))
    return nn.Sequential(nn.Conv2d(ni, nf, ks, padding=padding, stride=stride), nn.BatchNorm2d(nf))

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(ResnetBlock, self).__init__()
        self.expansion = 4
        self.conv1 = conv2d(in_channels, out_channels, ks=1, stride=1)
        self.conv2 = conv2d(out_channels, out_channels, ks=3, stride=stride)
        self.conv3 = conv2d(out_channels, out_channels * self.expansion, ks=1, stride=1)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
            
        x += identity
        x = self.relu(x)
        return x

In [ ]:
class OsicModel(torch.nn.Module):
    def __init__(self, ni, nh1, nh2, num_resnet_classes=[3, 4, 6, 3]):
        super(OsicModel, self).__init__()
        # Tab processing
        self.l1 = nn.Linear(ni, nh1)
        self.l1_bn = nn.BatchNorm1d(nh1, momentum=0.1)
        self.l2 = nn.Linear(nh1, nh2)
        
        # Image processing
        self.conv1 = conv2d(image_sample.shape[1], 128, ks=7, stride=2)
        self.maxpool1 = nn.MaxPool2d((3, 3), stride=2)
        # resnet layers
        self.in_channels = 64
        self.resnet1 = self.make_layer(num_resnet_classes[0], 128, stride=1)
        self.resnet2 = self.make_layer(num_resnet_classes[1], 128, stride=2)
        self.resnet3 = self.make_layer(num_resnet_classes[2], 256, stride=2)
        self.resnet4 = self.make_layer(num_resnet_classes[3], 512, stride=2)
        self.pool2d = nn.AdaptiveAvgPool2d(1)
        self.nh3 = 50
        self.ln1 = nn.Linear(2048 * 1 * 1, self.nh3)
        self.batchnorm = nn.BatchNorm1d(self.nh3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout2d(0.2)
        
        # Final layer
        self.p1 = nn.Linear(nh2 + self.nh3, 3)
        self.p2 = nn.Linear(nh2 + self.nh3, 3)
        
        
    def make_layer(self, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = conv2d(self.in_channels, out_channels*4, ks=1, stride=stride, padding=0)
            
        layers.append(ResnetBlock(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels * 4
        
        for i in range(num_residual_blocks - 1):
            layers.append(ResnetBlock(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)
        
    def forward(self, x, img):
        
        x = self.relu(self.l1(x))
        x = self.l1_bn(x)
        x = self.relu(self.l2(x))
        
        img = self.conv1(img)
        img = self.maxpool1(img)
        img = self.resnet1(img)
        img = self.resnet2(img)
        img = self.resnet3(img)
        img = self.resnet4(img)
        img = self.pool2d(img)
        img = img.reshape(img.shape[0], -1)
        img = self.ln1(img)
        img = self.batchnorm(img)
        img = self.relu(img)
        img = self.dropout(img)
        
        x = torch.cat((img, x), dim=1)
        
        p1 = self.p1(x)
        p2 = self.relu(self.p2(x))
        preds = p1 + torch.cumsum(p2, axis=1)
        return preds

In [ ]:
def create_model(nh1=100, nh2=100):
    model = OsicModel(X.shape[1], nh1, nh2, num_resnet_classes=[3, 4, 6, 3])
    model = model.to(device)
    wandb.config.hidden_layer_1 = nh1
    wandb.config.hidden_layer_2 = nh2
    wandb.config.hidden_layer_3 = model.nh3
    return model

In [ ]:
sample_model = create_model()

In [ ]:
criterion=mloss(0.8)

In [ ]:
# # Test model
y_sample, x_sample, image_sample = move_to_dev(y_sample, x_sample, image_sample)

In [ ]:
# img = sample_model.conv1(image_sample)
# img = sample_model.maxpool1(img)
# img = sample_model.resnet1(img)
# img = sample_model.resnet2(img)
# img = sample_model.resnet3(img)
# img = sample_model.resnet4(img)
# img = sample_model.pool2d(img)
# img = img.reshape(img.shape[0], -1)
# img = sample_model.ln1(img)
# img = sample_model.relu(img)
# img = sample_model.batchnorm(img)
# img = sample_model.dropout(img)
# img.shape

In [ ]:
# output = sample_model(x_sample, image_sample)
# criterion(y_sample, output)

#### Training functions

In [ ]:
EPOCHS=50
LR = 4e-3
wandb.config.epochs = EPOCHS
wandb.config.lr = LR

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
def eval_loop(valid_dl, model):
    with torch.no_grad():
        model.eval()
        total_eval_loss = 0
        total_eval_score = 0
        for x, y, image_tensor in valid_dl:
            x, y, image_tensor = move_to_dev(x, y, image_tensor)
            output = model(x, image_tensor)
            loss = criterion(y, output)
            total_eval_loss += loss.item()
            total_eval_score += score(y.unsqueeze(1), output)

        avg_val_loss = total_eval_loss / len(valid_dl)
        avg_val_score = total_eval_score / len(valid_dl) * -1
        return {
            'avg_val_loss': avg_val_loss,
            'avg_val_score': avg_val_score
        }

In [ ]:
def wandb_log(epoch, avg_train_loss, avg_val_loss, avg_val_score):
    wandb.log({'epoch': epoch, 'avg_train_loss': avg_train_loss, 'avg_val_loss': avg_val_loss, 'avg_val_score': avg_val_score})


def train_loop(epochs, train_dl, valid_dl, model, lr = 1e-3, print_score=False, model_name='test', use_wandb=False):
    steps = len(train_dl) * epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(train_dl), epochs=epochs)
    avg_train_losses = []
    avg_val_losses = []
    avg_val_scores = []
    scale_scores = []
    lr = []
    best_avg_val_score = -1000
    scaler = torch.cuda.amp.GradScaler() # mixed precision support
    for epoch in tqdm(range(epochs), total=epochs):
        model.train()
        total_train_loss = 0.0
        for i, (x, y, image_tensor) in enumerate(train_dl):
            x, y, image_tensor = move_to_dev(x, y, image_tensor)
            model.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(x, image_tensor)
                loss = criterion(y, output)
            total_train_loss += loss.item()
            
            # Backward Pass and Optimization
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            lr.append(get_lr(optimizer))
            scale_scores.append(scaler.get_scale())
        
        avg_train_loss = total_train_loss / len(train_dl)
        avg_train_losses.append(avg_train_loss)
        eval_res = eval_loop(valid_dl, model)
        avg_val_loss = eval_res['avg_val_loss']
        avg_val_score = eval_res['avg_val_score']
        avg_val_losses.append(avg_val_loss)
        avg_val_scores.append(avg_val_score.item())
        if use_wandb:
            wandb_log(epoch, avg_train_loss, avg_val_loss, avg_val_score)
        if best_avg_val_score < avg_val_score:
            best_avg_val_score = avg_val_score
            # save best model
            if os.path.isdir(path/'model') == False:
                os.makedirs(path/'model')
            torch.save(model.state_dict(), path/f'model/best_model_{model_name}.pt')
        if print_score:
            print(f'{epoch}: avg_val_score: {avg_val_score}')
    return pd.DataFrame({'avg_train_losses': avg_train_losses, 'avg_val_losses': avg_val_losses, 'avg_val_scores': avg_val_scores}), pd.DataFrame({'lr': lr}), pd.DataFrame({'scale_scores': scale_scores})

In [ ]:
res_df, lr_df, scale_df = train_loop(EPOCHS, sample_dl, sample_dl, sample_model, lr = LR)

In [ ]:
del sample_model
torch.cuda.empty_cache()

In [ ]:
res_df[['avg_train_losses', 'avg_val_losses']].plot()

In [ ]:
res_df[['avg_val_scores']].plot()

In [ ]:
lr_df.plot()

In [ ]:
scale_df.plot()

In [ ]:
res_df[['avg_val_scores']].max()

#### Training

In [ ]:
NFOLD = 5
kf = KFold(n_splits=NFOLD)

In [ ]:
def convert_to_tensor(df):
    return torch.tensor(df.values, dtype=torch.float32).to(device)

In [ ]:
submission_patients = submission_df['Patient']
submission_df['dummy_FVC'] = 0.0

In [ ]:
submission_dl = create_dl(submission_df[FE], pd.Series(np.zeros(submission_df[FE].shape[0])), submission_patients, image_path='test', shuffle=False)

In [ ]:
x_sample, y_sample, image_sample = next(iter(submission_dl))
x_sample.shape, y_sample.shape, image_sample.shape

In [ ]:
pe = np.zeros((submission_df.shape[0], 3))
pred = np.zeros((train_df.shape[0], 3))
pred.shape

In [ ]:
def predict(dl, model):
    prediction = []
    with torch.no_grad():
        model.eval()
        for x, y, image_tensor in dl:
            model = model.cpu()
            prediction.append(model(x.detach().cpu(), image_tensor.detach().cpu()))
    return torch.cat(prediction)

In [ ]:
%%time

res_dfs = []
for cnt, (tr_idx, val_idx) in tqdm(enumerate(kf.split(X)), total=NFOLD):
    X_train, y_train = X.loc[tr_idx], y[tr_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    print(f"FOLD {cnt}", X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
    model = create_model()
    train_dl = create_dl(X_train, y_train, train_df.loc[tr_idx]['Patient'], image_path='train')
    valid_dl = create_dl(X_valid, y_valid, train_df.loc[val_idx]['Patient'], image_path='train')
    res_df, _, _ = train_loop(EPOCHS, train_dl, valid_dl, model, print_score=True, lr = LR, model_name=str(cnt), use_wandb=True)
    res_dfs.append(res_df)
    pred[val_idx] = predict(valid_dl, model)
    del model
    torch.cuda.empty_cache()

In [ ]:
print("Mean validation score last:", np.mean([res_dfs[i]['avg_val_scores'][len(res_dfs[0]) - 1] for i in range(NFOLD)]))

In [ ]:
print("Best validation score:", np.mean([res_dfs[i]['avg_val_scores'].max() for i in range(NFOLD)]))

In [ ]:
from matplotlib.pyplot import figure

def plot_results(cols=['avg_train_losses', 'avg_val_losses']):
    nrows = len(res_dfs) // 2 + 1
    ncols = 2
    fig, axes = plt.subplots(nrows, ncols, figsize=(20, 10))
    figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
    for r in range(nrows):
        for c in range(ncols):
            index = r * 2 + c
            if index < len(res_dfs):
                res_dfs[r * 2 + c][cols].plot(ax=axes[r,c])
                
plot_results()

In [ ]:
plot_results(['avg_val_scores'])

#### Prediction

In [ ]:
def load_best_model(i):
    model_path = path/f'model/best_model_{i}.pt'
    model = create_model()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

In [ ]:
for i in range(NFOLD):
    model = load_best_model(i)
    pe += predict(submission_dl, model).numpy()
pe = pe / NFOLD

In [ ]:
sigma_opt = mean_absolute_error(y, pred[:, 1])
unc = pred[:,2] - pred[:, 0]
sigma_mean = np.mean(unc)
sigma_opt, sigma_mean

In [ ]:
submission_df['FVC1'] = pe[:,1]
submission_df['Confidence1'] = pe[:, 2] - pe[:, 0]

In [ ]:
submission_df.head(15)

In [ ]:
subm = submission_df[['Patient_Week','FVC','Confidence','FVC1','Confidence1']].copy()

In [ ]:
subm.loc[~subm.FVC1.isnull()].shape, subm.shape

In [ ]:
subm.loc[~subm.FVC1.isnull(),'FVC'] = subm.loc[~subm.FVC1.isnull(),'FVC1']

In [ ]:
if sigma_mean<70:
    subm['Confidence'] = sigma_opt
else:
    subm.loc[~subm.FVC1.isnull(),'Confidence'] = subm.loc[~subm.FVC1.isnull(),'Confidence1']

In [ ]:
subm.describe().T

In [ ]:
submission_df

In [ ]:
def replace_with_existing(df):
    for i in range(len(df)):
        patient_week_filter = subm['Patient_Week']==df.Patient[i]+'_'+str(df.Weeks[i])
        subm.loc[patient_week_filter, 'FVC'] = df.FVC[i]
        subm.loc[patient_week_filter, 'Confidence'] = 0.1

train_df = pd.read_csv(path/'train.csv', dtype = common.TRAIN_TYPES)
test_df = pd.read_csv(path/'test.csv', dtype = common.TRAIN_TYPES)
replace_with_existing(train_df)
replace_with_existing(test_df)

In [ ]:
subm[subm['Patient_Week'].str.find('ID00419637202311204720264') > -1].head(30)

In [ ]:
subm[["Patient_Week","FVC","Confidence"]].to_csv("submission.csv", index=False)

In [ ]:
submission_final_df = pd.read_csv('submission.csv')

In [ ]:
test_df['Patient'].unique()

In [ ]:
submission_final_df[submission_final_df['Patient_Week'].str.find('ID00419637202311204720264') == 0]['FVC'].plot()

In [ ]:
submission_final_df[submission_final_df['Patient_Week'].str.find('ID00421637202311550012437') == 0]['FVC'].plot()

In [ ]:
submission_final_df[submission_final_df['Patient_Week'].str.find('ID00423637202312137826377') == 0]['FVC'].plot()

In [ ]:
submission_final_df[submission_final_df['Patient_Week'].str.find('ID00422637202311677017371') == 0]['FVC'].plot()

In [ ]:
!cat submission.csv